<a href="https://colab.research.google.com/github/milanfx/DTSA-5511/blob/main/NLP_Disaster_Tweets_Kaggle_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="font-family:Roboto Condensed SemiBold; font-size:150%; letter-spacing: 0px; background-color:darkgreen; color:#f8f2e4; text-align:center; padding-top:10px; padding-bottom:10px; border: 5px solid #f8f2e4; border-radius: 20px;"><h1><b>
NLP Disaster Tweets Kaggle Mini-Project
</b></h1></div>

---

<div style="text-align:center">
<img src="https://github.com/milanfx/Dataset/blob/main/images/nlp_logo.png?raw=true" style="width:850px; height:200px;">
</div>


<div style="font-family:Roboto Condensed SemiBold; font-size:120%; letter-spacing: 0px; background-color:#f8f2e4; color:darkgreen; text-align:center; padding-top:0px; padding-bottom:0px; border: 3px solid darkgreen; border-radius: 20px;">

## **1. Project Introduction**

</div>

---

### **1.1 Project understanding**

**Background:**

-


**Objective:**

-


**Chanllenge:**

-

---

### **1.2 Data Understanding**

**Data Description**

-


**Evaluation Method**

-


---

### **1.3 Data Loading**

**Load Library:**

**Load Data from Kaggle:**

In [ ]:
from google.colab import userdata

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle competitions download -c
!unzip -q  -d dataset

100% 6.31G/6.31G [05:19<00:00, 23.5MB/s]
100% 6.31G/6.31G [05:19<00:00, 21.2MB/s]


**Check Data Size:**

In [ ]:
train_path = 'dataset/train/'
test_path = 'dataset/test/'

train_image = glob(os.path.join(train_path, '*.tif'))
test_image = glob(os.path.join(test_path, '*.tif'))

print(f'Number of images in train: {len(train_image)}')
print(f'Number of images in test: {len(test_image)}')

Number of images in train: 220025
Number of images in test: 57458


-

---

<div style="font-family:Roboto Condensed SemiBold; font-size:120%; letter-spacing: 0px; background-color:#f8f2e4; color:darkgreen; text-align:center; padding-top:0px; padding-bottom:0px; border: 3px solid darkgreen; border-radius: 20px;">

## **2. Exploratory Analysis**

</div>

---

### **2.1 Sample Distribution**

**Check the Proportion of Negative and Positive samples:**

In [1]:
labels_path = 'dataset/train_labels.csv'
sub_path = 'dataset/sample_submission.csv'

train_data = pd.read_csv(labels_path)
test_data = pd.read_csv(sub_path)

plt.figure(figsize=(5, 3))
train_data['label'].value_counts().plot(kind='bar')
plt.title('Distribution of Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

NameError: name 'pd' is not defined

In [ ]:
train_data['label'].value_counts()

- The ratio of positive to negative samples stands at 4:6; thus, rebalancing may be considered, though it is not required.

---

### **2.2 Positive Samples**

**Preview 15 Positive Images:**

In [2]:
positive_train_file_name = train_data[train_data['label'] == 1]['id']
negative_train_file_name = train_data[train_data['label'] == 0]['id']

NameError: name 'train_data' is not defined

In [3]:
fig, ax = plt.subplots(figsize=(9, 6))

count = 1
for index, name in enumerate(positive_train_file_name.head(15)):
    file_name = os.path.join(train_path, f"{name}.tif")
    img = Image.open(file_name)
    ax = plt.subplot(3, 5, count)
    ax.imshow(np.array(img))
    ax.axis('off')
    count += 1
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

---

### **2.3 Negative Samples**

**Preview 15 Negative Images:**

In [4]:
fig, ax = plt.subplots(figsize=(9, 6))

count = 1
for index, name in enumerate(negative_train_file_name.head(15)):
    file_name = os.path.join(train_path, f"{name}.tif")
    img = Image.open(file_name)
    ax = plt.subplot(3, 5, count)
    ax.imshow(np.array(img))
    ax.axis('off')
    count += 1
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

- For non-professionals, positive and negative images appear the same.

---

<div style="font-family:Roboto Condensed SemiBold; font-size:120%; letter-spacing: 0px; background-color:#f8f2e4; color:darkgreen; text-align:center; padding-top:0px; padding-bottom:0px; border: 3px solid darkgreen; border-radius: 20px;">

## **3. Model Architecture**

</div>

---

### **3.1 Split Train and Validation**

**Normalize and Shuffle the Training Data, then Split into 80% and 20%:**

In [5]:
train_data["filename"] = train_data.id + '.tif'
test_data['filename'] = test_data.id + '.tif'

train_data['label'] = train_data['label'].astype(str)
train_data = shuffle(train_data, random_state=1)

NameError: name 'train_data' is not defined

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.2)

test_datagen = ImageDataGenerator(
    rescale=1./255)

NameError: name 'ImageDataGenerator' is not defined

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_path,
    x_col="filename",
    y_col="label",
    subset="training",
    batch_size=32,
    seed=1,
    class_mode="binary",
    target_size=(96,96))

NameError: name 'train_datagen' is not defined

In [8]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_path,
    x_col="filename",
    y_col="label",
    subset="validation",
    batch_size=32,
    seed=1,
    class_mode="binary",
    target_size=(96,96))

NameError: name 'train_datagen' is not defined

In [9]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_path,
    x_col="filename",
    y_col=None,
    batch_size=32,
    seed=1,
    class_mode=None,
    shuffle=False,
    target_size=(96,96))

NameError: name 'test_datagen' is not defined

-

---

### **3.2 Classic RNN Model**

**The model is**

-

In [10]:
classic_cnn = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(96, 96, 3)),
    MaxPooling2D(2, 2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(units=512, activation='relu'),
    Dropout(rate=0.3),
    Dense(1, activation='sigmoid')
], name='classic_cnn')

classic_cnn.summary()

NameError: name 'Sequential' is not defined

- This model contains over 6 million parameters, yet its complexity remains within manageable limits.

In [11]:
optimizer = Adam(learning_rate=0.0005)
metrics=['accuracy', AUC(name='auc')]
early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=2, verbose=1, restore_best_weights=True)

classic_cnn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

history = classic_cnn.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[early_stopping])

NameError: name 'Adam' is not defined

-

<div style="font-family:Roboto Condensed SemiBold; font-size:120%; letter-spacing: 0px; background-color:#f8f2e4; color:darkgreen; text-align:center; padding-top:0px; padding-bottom:0px; border: 3px solid darkgreen; border-radius: 20px;">

## **4. Results and Analysis**

</div>

---

### **4.1 Model Performance**

**Accuracy:**

In [12]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

NameError: name 'plt' is not defined

**Loss:**

In [13]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

NameError: name 'plt' is not defined

**AUC:**

In [14]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['auc'], label='Train AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

NameError: name 'plt' is not defined

-


**Confusion Marix:**

In [15]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_path,
    x_col="filename",
    y_col="label",
    subset="validation",
    batch_size=32,
    seed=1,
    class_mode="binary",
    target_size=(96,96),
    shuffle=False)
valid_generator.reset()

predictions = classic_cnn.predict(valid_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)
actual_classes = valid_generator.classes

cm = confusion_matrix(actual_classes, predicted_classes.flatten())

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Cancer', 'Cancer'], yticklabels=['Non-Cancer', 'Cancer'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

NameError: name 'train_datagen' is not defined

- Based on the confusion matrix, we can observe a balanced distribution of misclassified samples.

---

### **4.2 Prediction Accuracy**

In [16]:
predictions = classic_cnn.predict(test_generator).flatten()

result = pd.DataFrame({
    "id": test_data['id'],
    "label": predictions
})
result.to_csv('Classic CNN.csv', index=False)

NameError: name 'classic_cnn' is not defined

<div style="text-align:center">
<img src="" style="width:850px;">
</div>


-

---

<div style="font-family:Roboto Condensed SemiBold; font-size:120%; letter-spacing: 0px; background-color:#f8f2e4; color:darkgreen; text-align:center; padding-top:0px; padding-bottom:0px; border: 3px solid darkgreen; border-radius: 20px;">

## **5. Project Conclusion**

</div>

---

### **5.1 Model Summary**

1.

2.

3. The shortcomings of this model include:

-

---

### **5.2 Next Step**

1.

2.

3.

---